# Problem 3

In this problem, you will "receive" a stream of samples that you need to demodulate. If you succeed, the decoded message conatins an image that you can view as a reward.

You should use this notebook as the starting point for your work.

This document is fairly long because it contains some code to make your work easier and because I want to make sure that you can see exactly how the transmitter works. You should step through this code before you attempt to demodulate as there are also a few variables and definitions that are set along the way.

In a nut-shell, the received samples encode nearly 200,000 bytes of image data. They are transmitted over multiple burst that are contained in the received samples that you will load into this notebook.

Each burst contains a preamble, a fixed-length header, and a variable length payload. The payload is modulated using one of three possible modulation formats; the header always uses BPSK. The format of the header and functions for reading the header are given below.

There is a variable size gap between consecutive bursts. The channel adds noise and introduces a an amplitude, phase, and frequency shift; the the frequency offset is slowly time-varying.

More details are provided as you read through the notebook.



In [3]:
## Boilerplate instructions for importing NumPy and Matplotlib
# Import NumPy
import numpy as np

import struct # for header

# To plot pretty figures, use matplotlib
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

## Helper functions

The following functions are from notebooks used in class. They should look familiar. Make sure that they have all been "run" so that you can use them for your work.

In [4]:
def srrc_pulse(a, fsT, N=5):
    """Construct a raised cosine pulse
    
    Inputs:
    a - roll-off factor
    fsT - number of samples per symbol period
    N - lenght of pulse in symbol periods; pulse ranges for -N \leq t/T \leq N (default: 5).

    Returns:
    Length 2*N*fsT+1 vector
    """
    # time axis with spacing 1/(fs*T)
    tt = np.linspace(-N, N, 2*N*fsT + 1)
    
    num = np.sin(np.pi*tt*(1-a)) + 4*a*tt*np.cos(np.pi*tt*(1+a))
    den = np.pi*tt*(1-(4*a*tt)**2)
    
    # deal with divide-by-zeros: at zero location, place "L'Hospital value" in numerator
    # and 1 in denominator.
    # First divide-by-zero location is t=0; by L-Hospital, the value is (1 + a*(4/pi - 1))
    ind_0 = np.where(np.abs(tt) < 1e-6)
    num[ind_0] = (1 + a*(4/np.pi - 1))
    den[ind_0] = 1
    # Second divide-by-zero location is t=+/-1/(4*a); by L-Hospital, the value is as shown below
    ind_0 = np.where(np.abs(np.abs(tt) - 1/(4*a)) < 1e-6)
    num[ind_0] = a/np.sqrt(2) * ((1+2/np.pi)*np.sin(np.pi/(4*a)) + (1-2/np.pi)*np.cos(np.pi/(4*a)))
    den[ind_0] = 1
    
    # scaling: we scale the pulse such that the convolution of two SRRC pulse yields
    # a RC pulse with amplitude 1 in the center of the pulse. This implies that
    # np.sum(hh**2) must equal 1. This replace the scaling by 1/T in the formula above.
    hh = num/den
    
    return hh / np.sqrt(np.sum(hh*hh))

In [5]:
def pulse_shape(symbols, pulse, fsT):
    """Generate a pulse-shaped QAM signal
    
    Inputs:
    symbols - a sequence of information symbols; rate 1/T
    pulse - sampled pulse shape; rate fsT/T
    fsT - samples per symbol period

    Returns:
    Numpy array with fsT*(len(symbols) - 1) + len(pulse)  samples
    """
    # step 1: upsample the symbol sequence
    up_symbols = np.zeros(fsT * (len(symbols) - 1) + 1, dtype=symbols.dtype)
    up_symbols[::fsT] = symbols

    # step 2: filter
    return np.convolve(up_symbols, pulse)

This version of the MPE decision rule is slightly different; it returns the index of the best symbol - not the symbol itself

In [6]:
def MPE_decision_rule(Z, A):
    """Decide which symbol was most likely transmitted
    
    This function examines matched filter outputs (in vector Z) and for each element of Z selects the symbol 
    from constellation A that is closest.

    Inputs:
    Z - Vector of matched filter outputs
    A - Constellation

    Returns:
    Numpy array of the same length as Z
    """
    #dec = np.empty_like(Z)
    ind = np.empty_like(Z, dtype=np.uint8)

    for n in range(len(Z)):
        this_Z = Z[n]
        ind[n] = np.argmin(np.abs(A-this_Z))
        # dec[n] = A[ind[n]]

    return ind

### Functions for converting between byte sequences and symbol sequences

The transmitted information is a sequence of bytes. However, transmission occurs in symbols that can carry `bps` bits per symbol.

The following two functions convert between bytes and symbol indices. More precisely, between base $2^8$ for bytes and base $2^{bps}$ for symbols. 

You will see how they are used in both the header and the transmitter.

In [7]:
def bytes_to_symbols(msg, bps):
    """Convert a sequence of bytes to symbols
    
    Inputs:
    msg - a vector of bytes
    bps - bits per symbol
    
    Returns:
    vector of ints (all ints are between 0 and 2**bps - 1)
    
    Note: only works for bps in [1, 2, 4]
    """
    
    syms_per_byte = 8 // bps
    mask = np.uint8(2**bps - 1)
    
    out = np.zeros(syms_per_byte * len(msg), dtype=np.uint8)

    for n in range(len(msg)):
        b = msg[n]
        for m in range(syms_per_byte):
            out[n*syms_per_byte + m] = (b >> (m*bps)) & mask
            
    return out

In [8]:
bytes_to_symbols(b'Hello', 4)

array([ 8,  4,  5,  6, 12,  6, 12,  6, 15,  6], dtype=uint8)

In [9]:
def symbols_to_bytes(syms, bps):
    """Convert a sequence of symbols to bytes
    
    Inputs:
    syms - a vector of bytes
    bps - bits per symbol
    
    Returns:
    vector of uint8 (aka char)
    
    Note: only works for bps in [1, 2, 4]
    """
    syms_per_byte = 8 // bps
    
    msg = np.zeros(len(syms) // syms_per_byte, dtype=np.uint8)
    
    for n in range(len(msg)):
        for m in range(syms_per_byte):
            msg[n] += syms[n*syms_per_byte + m] << (m*bps)
            
    return msg

In [10]:
# round-trip test
symbols_to_bytes( bytes_to_symbols(b'Hello', 4), 4).tobytes()

b'Hello'

## Signal Definitions

We are now ready to dive into the details of the transmitted signal, beginning with the three modulation formats use in the transmitter.

### Constellations

We will use three different constellations: BPSK, QPSK, and 16-QAM. For each, we define a dictionary that shows the mapping of bit-patterns to symbols.

For each of the three constellations, we have:
* a dictionary that shows the mapping from bit-patterns to symbols
* an array that list the symbols in the order indicated by the bit-pattern; e.g., the symbol for `0b0000` (decimal 0) is first, followed by the symbol for `0b0001` (decimal 1).
* an integer that is used as a unique code to refere to the constellation.



In [11]:
BPSK_d = {
    0b0: 1.  + 0j,
    0b1: -1. + 0j
}

# normalized alphabet in order
A_BPSK = np.array([BPSK_d[n] for n in range(2)])

BPSK_MOD_CODE = 0

In [12]:
QPSK_d = {
    0b00:  1. + 1.j,
    0b01: -1. + 1.j,
    0b11: -1. - 1.j,
    0b10:  1. - 1j
}

# normalized alphabet in order
A_QPSK = np.array([QPSK_d[n] for n in range(4)])/np.sqrt(2)

QPSK_MOD_CODE = 1

In [13]:
QAM16_d = {
    0b0000: -3. + 3.j,
    0b0100: -1. + 3.j,
    0b1100:  1. + 3.j,
    0b1000:  3. + 3.j,
    0b0001: -3. + 1.j,
    0b0101: -1. + 1.j,
    0b1101:  1. + 1.j,
    0b1001:  3. + 1.j,
    0b0011: -3. - 1.j,
    0b0111: -1. - 1.j,
    0b1111:  1. - 1.j,
    0b1011:  3. - 1.j,
    0b0010: -3. - 3.j,
    0b0110: -1. - 3.j,
    0b1110:  1. - 3.j,
    0b1010:  3. - 3.j,
}

# normalized alphabet in order
A_QAM16 = np.array([QAM16_d[n] for n in range(16)])/np.sqrt(4*2 + 8*10 + 4*18)

QAM16_MOD_CODE = 2

This dictionary below collects all three modulations into a single table. It is used by the transmitter to look up details of the constellation in use for a given frame. 

The inverse table `rev_mod_table` in particular is useful to translate between numerical `mod_codes` and the constellation names that are keys into the table.

These tables are used by the trasnmitter. You will need them for your receiver as well.

In [14]:
mod_table = {
    'BPSK': {
        'name': 'BPSK',
        'mod_code': BPSK_MOD_CODE,
        'bps': 1,
        'alphabet': A_BPSK,
        'map': BPSK_d,
    },
    'QPSK': {
        'name': 'QPSK',
        'mod_code': QPSK_MOD_CODE,
        'bps': 2,
        'alphabet': A_QPSK,
        'map': QPSK_d,
    },
    'QAM16': {
        'name': '16-QAM',
        'mod_code': QAM16_MOD_CODE,
        'bps': 4,
        'alphabet': A_QAM16,
        'map': QAM16_d,
    }
}

# make an inverse table for looking up names by mod_code
rev_mod_table = {}
for k,v in mod_table.items():
    rev_mod_table[v['mod_code']] = k

### Preamble

The preamble is a known pattern of BPSK symbols. There is not much ealse to say. The preamble pattern is shown below:

In [15]:
preamble_seq = np.array([-1.,  1.,  1.,  1., -1.,  1.,  1., -1., 
                         -1.,  1., -1.,  1., -1., -1., -1.])

### Header 

Information is transmitted in bursts that vary in size and that may be using different modulation formats. To support this feature, the frames contain a *header* that is located between the preamble and the *payload* portion of the burst.

The header is always the same length and it is always BPSK modulated. It contains the following fields:
* `seq`: a sequence number that is incremented from one burst to the next. You can use this to determine if you missed a burst (8-bit unsigned integer, rolls over at 256)
* `mod_code`: an integer that indicates the modulation format for the *payload*. Permitted values are 0, 1, 2 corresponding to the three modulation formats given above, e.g., 0 indicates BPSK. (8-bit unigned integer)
* `pld_n_syms`: number of symbols in the payload. (16-bit unsigned int)

The header always contains 32 bits.

The following two functions, encode and decode the header. Specifically, they translate to and from three integers and an array of 32 bits (stored as uint8).

In [16]:
def encode_hdr(seq, mod_code, pld_n_syms):
    """encode the header
    
    Inputs:
    seq - sequence number
    mod_code - integer indicating modulation for payload
    pld_n_syms - number of symbols in the payload
    
    Returns:
    NumPy array of 32 unsigned 
    
    """
    hdr_bytes = struct.pack('BBH', seq % 256, mod_code, pld_n_syms)
                
    return bytes_to_symbols(hdr_bytes, 1)

In [17]:
# try it out
encode_hdr(1,2,3)

array([1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=uint8)

In [18]:
def decode_hdr(bit_vec):
    """decode a vector of header bits
    
    Input:
    bit_vec - vector of bits to be decoded
    
    Returns:
    (seq, mod_code, pld_n_syms) tuple
    
    """
    n_bytes = struct.calcsize('BBH')
    assert len(bit_vec) == 8*n_bytes
    
    tmp = symbols_to_bytes(bit_vec, 1)
            
    return struct.unpack('BBH', tmp.tobytes())

In [20]:
# round-trip: encode, then decode
decode_hdr( encode_hdr(12, 1, 12345) )

(12, 1, 12345)

### Transmitter

We now have everything we need to constructs signal bursts (or frames) from a given message of bytes. There are only a few remaining parameters we need to fix.

The received signal is oversampled `fst = 8` times. The transmitter always uses pulse shaping with SRRC pulses with roll-off factor equal to 0.5.

In [21]:
fsT = 8
alpha = 0.5
hh = srrc_pulse(alpha, fsT)

The following function generates transmitted samples from a sequence of bytes. It is the main function of the transmitter and includes both the preamble as well as the header.

If you have any lingering questions, this function should clarify them.

In [22]:
def transmit_burst(msg_bytes, seq, mod_code):
    """generate samples from a sequence of bytes
    
    msg_bytes - a vector of bytes that are to be transmitted
    seq - a sequence number; the frames in the received message are numbered uniquely
    mod_code - the code that indicates how the payload is modulated
    
    Returns:
    A vector of IQ samples
    """
    
    # look up the modulation format
    mod_name = rev_mod_table[mod_code]
    mod_data = mod_table[mod_name]
    A = mod_data['alphabet']
    
    # convert msg_bytes to modulated symbols 
    syms = bytes_to_symbols(msg_bytes, mod_data['bps'])
    mod_syms = A[syms]
    
    # construct the header and turn it into BPSK symbols
    hdr = encode_hdr(seq, mod_code, len(syms))
    mod_hdr = A_BPSK[hdr]
    
    # concatenate preamble, header, and payload
    frame = np.concatenate((preamble_seq, mod_hdr, mod_syms))
    
    # only thing left is pulse shaping
    return pulse_shape(frame, hh, fsT)

### Recovering a burst

The round-trip test below shows that a message can be receovered from transmitted samples. It is not intended to be a working receiver, but it demonstrates that the message bytes can be recovered and how to do so.

In [174]:
# round-trip test - no noise, no nothing
# The transmitted message is just 'Hello'
samples = transmit_burst(b'Hello', 0, 2)

# received starts here
# first, pass signal through a matched filter
mf_out = np.convolve(samples, hh)

# down-sample, as there is no delay we know the right sampling phase
Z = mf_out[len(hh)-1::fsT]

# check that the preamble matches, preamble is 15 symbols long
assert np.abs(np.sum(Z[:15] - preamble_seq)) < 0.1

# then, demod the header - the 32 symbols after the preamble
hdr_Z = Z[15:47]
hdr_syms = MPE_decision_rule(hdr_Z, A_BPSK)
seq, mod_code, pld_len = decode_hdr(hdr_syms)
print("(seq, mod_code, pld_len) = ", seq, mod_code, pld_len)

# now the payload
# we learned how long it is (see pld_len below)
pld_Z = Z[47 : 47+pld_len]
# and how it's modulated (see mod_code below)
# use that to look up pertinent info about the modulation
mod_name = rev_mod_table[mod_code]
mod_data = mod_table[mod_name]
A = mod_data['alphabet']

# hard decision
pld = MPE_decision_rule(pld_Z, A)

# print the decode message
print("Message:", symbols_to_bytes(pld, mod_data['bps']).tobytes())


(seq, mod_code, pld_len) =  0 2 10
Message: b'Hello'


## Load the data

That's it. Now it's time to load the received samples and for you to demodulate them.

You can check that you received all frames by lookin at the sequence numbers. If all works well, you should end up with 187,500 (375 x 500) bytes.

These bytes represent a grayscale image that you can display with the command:
```
plt.imshow(img_seq.reshape(375, 500), cmap='gray', vmax=255)
```
where `img_seq` is the byte sequence that you recovered.

Ok, here is the command to load the samples. The variable `rr` holds the entire sequence of samples - all 7,231,013 of them.

In [35]:
import requests
import io

response = requests.get('https://www.dropbox.com/s/u1wqgnwhfy3x2g9/samples.npy?dl=1')
response.raise_for_status()
rr = np.load(io.BytesIO(response.content), allow_pickle=True) 


In [36]:
# This should produce:
# array([ 0.01417607+0.01527566j, -0.00072523+0.04899158j,
#        -0.00179149-0.02089944j, -0.02164281+0.03086397j,
#        -0.00588557-0.01808985j,  0.08452638-0.04749704j,
#        -0.01872568-0.01456672j,  0.11407796-0.0319316j ,
#        -0.0077451 +0.02264611j, -0.00696297-0.00700862j])

rr[:10]

array([ 0.01417607+0.01527566j, -0.00072523+0.04899158j,
       -0.00179149-0.02089944j, -0.02164281+0.03086397j,
       -0.00588557-0.01808985j,  0.08452638-0.04749704j,
       -0.01872568-0.01456672j,  0.11407796-0.0319316j ,
       -0.0077451 +0.02264611j, -0.00696297-0.00700862j])